## Coding Exercise #0511

**TfidfVectorizer**

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer


In [1]:
import numpy as np
import re
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
warnings.filterwarnings('ignore')               # Turn the warnings off.
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### 1. NLP classification analysis:

#### 1.1. Read in data:
Movie review data from Cornell CS department. <br>
- The data can be found [here](https://www.cs.cornell.edu/people/pabo/movie-review-data). <br>
- Download the “polarity dataset” and unzip. <br>
- Under the "txt_sentoken” folder, there are “pos” and “neg" subfolders. <br>

In [2]:
#You can save the dataset on your drive folder, and mount it
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Specify the folder and read in the subfolders.
reviews = load_files('/content/drive/MyDrive/txt_sentoken')
#reviews = load_files('txt_sentoken/')
my_docs, y = reviews.data, reviews.target

In [4]:
print(len(my_docs)," ",len(y))

2000   2000


In [5]:
np.unique(y, return_counts=True)

(array([0, 1]), array([1000, 1000]))

#### 1.2. Data pre-processing:

In [6]:
corpus = []
stop_words = set(stopwords.words('english'))

for i in range(0, len(my_docs)):
    review = re.sub(r'\W', ' ', str(my_docs[i]))
    review = review.lower()
    review = re.sub(r'^br$', ' ', review)
    review = re.sub(r'\s+br\s+',' ',review)
    review = re.sub(r'\s+[a-z]\s+', ' ',review)
    review = re.sub(r'^b\s+', '', review)
    review = re.sub(r'\s+', ' ', review)               # Remove the excess white spaces introduced in the previous steps.
    review = [word for word in word_tokenize(review) if word not in stop_words]
    corpus.append(' '.join(review))

In [ ]:
# Just show a document.
corpus[0]

#### 1.3. Create a embedding Word2Vec representation:

In [10]:
# Cargar el modelo Word2Vec preentrenado
import gensim
from nltk.data import find
nltk.download('word2vec_sample')
# Cargar el modelo de embeding pre-entrenados del NLTK
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)



[nltk_data] Downloading package word2vec_sample to /root/nltk_data...
[nltk_data]   Package word2vec_sample is already up-to-date!


In [19]:
# Entrenar un Word2Vec
from gensim.models import Word2Vec

reviews = [review.split() for review in corpus]
model = Word2Vec(reviews, vector_size=100, epochs=10)

In [27]:
def vectorize(sentence):
  words = sentence.split() # the sentence has been previoulsy tokenized in the preprocess
  words_emb = [model.wv[word] for word in words if word in model.wv]
  #words_emb = [model[word] for word in words if word in model.key_to_index] #for KeyedVectors
  if len(words_emb) == 0:
    return np.zeros(300)
  words_emb = np.array(words_emb)
  return words_emb.mean(axis=0)

In [23]:
X = np.array([vectorize(review) for review in corpus])  #Transforms Documents to document-embedding matrix
X.shape

(2000, 100)

#### 1.4. Logistic regression training and testing:

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1234)

In [25]:
# Training.
LR = LogisticRegression()
LR.fit(X_train, y_train)

LogisticRegression()

In [26]:
# Testing.
y_pred = LR.predict(X_test)
conf_mat = confusion_matrix(y_test, y_pred)
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Precision: ", precision_score(y_test, y_pred))
print("Recall: ", recall_score(y_test, y_pred))
print("F1_score: ", f1_score(y_test, y_pred))

Accuracy:  0.7566666666666667
Precision:  0.7524752475247525
Recall:  0.7625418060200669
F1_score:  0.7574750830564784


In [18]:
# Confusion matrix.
#Thus in binary classification, the count of true negatives is
#C_{0,0}, false negatives is C_{1,0}, true positives is
#C_{1,1} and false positives is C_{0,1}.
#    neg pos
#neg
#pos
print(conf_mat)

[[220  81]
 [ 78 221]]
